## Install invasive species mapping repo

In [1]:
%pip install watermark geemap geeml -q
!git clone https://github.com/Geethen/Invasive_Species_Mapping.git
import sys
sys.path.insert(0,'/content/Invasive_Species_Mapping/code')

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip
fatal: destination path 'Invasive_Species_Mapping' already exists and is not an empty directory.


In [2]:
%load_ext watermark

In [3]:
import ee
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()
import geemap

# Add module to environment varibles
import sys
MODULE_FULL_PATH = r'C:\Users\coach\myfiles\postdoc\code\Invasive_Species_Mapping\code'
sys.path.insert(1, MODULE_FULL_PATH)

# Load python modules with preprocessing functions. 
from timeSeriesFunctions import prepareTS
from covariateFunctions import prepareCovariates
from trainDataFunctions import prepareTrainingData
from modelFitFunctions import prepareModel
from metricFunctions import prepareMetrics
from geeml.utils import eeprint

In [4]:
%watermark -v -m --iversions

Python implementation: CPython
Python version       : 3.9.13
IPython version      : 8.4.0

Compiler    : MSC v.1929 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 165 Stepping 2, GenuineIntel
CPU cores   : 12
Architecture: 64bit

sys   : 3.9.13 | packaged by conda-forge | (main, May 27 2022, 16:50:36) [MSC v.1929 64 bit (AMD64)]
ee    : 0.2
geemap: 0.17.1



## Import DynamicWorld validation

In [5]:
af = ee.ImageCollection("projects/sat-io/open-datasets/LandCoverNet/LABELS/ref_landcovernet_af_v1_labels")

In [6]:
# define your study area
study_area1 = af.limit(10).geometry(100).bounds(100)
Map = geemap.Map()
Map.addLayer(study_area1, {}, 'study_area1')
Map.centerObject(study_area1, 3)
Map

Map(center=[17.595927441287667, -15.532191252620184], controls=(WidgetControl(options=['position', 'transparen…

In [8]:
# start and end dates for time series
Date_Start = ee.Date('2018-01-01')
Date_End = ee.Date('2019-01-01')
# how many days to summarise in each image e.g 30 days = 12 images per year
day_int = 366 #step size
# cloud probabillity threshold
CLOUD_THRESH=30
# which bands to keep
BANDSS2 = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6','B7', 'B8', 'B8A', 'B9', 'B11','B12', 'ndvi']

## Prepare covariates/composites

In [9]:
s21cTS = prepareTS(Date_Start,Date_End,day_int,study_area1,CLOUD_THRESH,BANDSS2).timeSeries(satellite = 'S2', s2_level = 1)
s21cTS

In [10]:
s22aTS = prepareTS(Date_Start,Date_End,day_int,study_area1,CLOUD_THRESH,BANDSS2).timeSeries(satellite = 'S2', s2_level = 2)
s22aTS

In [11]:
# export training data and image to classify
# Define the export parameters

export_params1 = {
    'image': s21cTS,
    'description': 'export_inferenceImage',
    'assetId': 'projects/ee-geethensingh/assets/UQ/s21c_inferenceImage',
    'scale': 10,
    'region': study_area1,
    'maxPixels': 1e13
}

export_params2 = {
    'image': s22aTS,
    'description': 'export_inferenceImage',
    'assetId': 'projects/ee-geethensingh/assets/UQ/s22a_inferenceImage',
    'scale': 10,
    'region': study_area1,
    'maxPixels': 1e13
}

# Export the data to the Earth Engine asset
task1 = ee.batch.Export.image.toAsset(**export_params1)
task1.start()
# task2 = ee.batch.Export.image.toAsset(**export_params2)
# task2.start()

## Extract labels and covariates from composite

In [11]:
# name of the label property
label = 'label'

afsubset = af.select([0], ['label']).limit(10).mosaic()
# Generate stratified sample points
train = s21cTS.addBands(afsubset).stratifiedSample(numPoints=10, classBand='label', region= study_area1,
scale= 10, seed=42, geometries= True)


In [13]:
# export training data 
# Define the export parameters
export_params1 = {
    'collection': train,
    'description': 'export_trainData',
    'assetId': 'projects/ee-geethensingh/assets/UQ/LandCoverNet_train',
}
# Initiate export the data to the Earth Engine asset
task = ee.batch.Export.table.toAsset(**export_params1)
task.start()

In [12]:
bandnames = s21cTS.bandNames()
# number of classes
NCLASS = train.aggregate_array('label').distinct().length().getInfo()

EEException: User memory limit exceeded.

In [ ]:
model = ee.Classifier.smileRandomForest(40).setOutputMode('MULTIPROBABILITY').train(
  features = points,
  classProperty = label,
  inputProperties = bandnames
)

In [17]:
# Generate probability output
classes = points.aggregate_array('label').distinct().map(lambda value: ee.Algorithms.String(value))
probImage = s21cTS.classify(model).arrayFlatten([classes])

## Uncertainty Quantification

In [14]:
%load_ext autoreload

In [18]:
# Generate calibration points (uses different seed)
calibration =  probImage.addBands(afsubset).stratifiedSample(numPoints=10, classBand='label', region= study_area1,
scale= 250, seed = 43, geometries= True)
# Generate test points (uses different seed)
test =  probImage.addBands(afsubset).stratifiedSample(numPoints=10, classBand='label', region= study_area1,
scale= 250, seed = 44, geometries= True)

In [21]:
calibration.aggregate_array('label').distinct()

In [22]:
test.aggregate_array('label').distinct()

In [23]:
classes

In [20]:
%autoreload 3

from conformalClassifier import conformalClassifier
cc = conformalClassifier(probImage, calibration, test, label, 0.2)
conformalMasks = cc.quantifyImageUncertainty()
evalStats = cc.evaluateUncertainty()

Average set size: 0.631578947368421


EEException: Computation timed out.